In [7]:
import requests
from bs4 import BeautifulSoup
import datetime

# 获取页面
def get_page(link):
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6'} 
    r = requests.get(link, headers = headers)
    html = r.content  #使用r.content解封装
    html = html.decode('utf-8')  #由UTF-8解码为unicode
    soup = BeautifulSoup(html, 'lxml')
    return soup

# 解析网页
def get_data(post_list):
    data_list =[]
    for post in post_list:
        title = post.find('div',class_='titlelink box').text.strip()
        post_link = post.find('div',class_='titlelink box').a['href']
        post_link = "https://bbs.hupu.com" + post_link 

        author = post.find('div',class_='author box').a.text.strip()
        author_page = post.find('div',class_='author box').a['href']
        start_date = post.find('div',class_='author box').contents[5].text.strip()

        reply_view = post.find('span',class_='ansour box').text.strip()
        reply = reply_view.split('/')[0].strip()
        view = reply_view.split('/')[1].strip()

        reply_time = post.find('div',class_='endreply box').a.text.strip()
        last_reply = post.find('div',class_='endreply box').span.text.strip()
        if ':' in reply_time: #时间是11:27
            date_time = str(datetime.date.today()) + ' ' + reply_time
            date_time = datetime.datetime.strptime(date_time, '%Y-%m-%d %H:%M')
        elif reply_time.find("-") == 4: #时间是2017-02-27
            date_time = datetime.datetime.strptime(reply_time, '%Y-%m-%d').date()
        else: #时间是11-27
            date_time = datetime.datetime.strptime('2018-' + reply_time, '%Y-%m-%d').date()
        data_list.append([title, post_link, author, author_page, start_date, reply, view, last_reply, date_time])
    return data_list

link = "https://bbs.hupu.com/bxj"
soup = get_page(link)
post_all= soup.find('ul', class_="for-list")
post_list = post_all.find_all('li')
data_list = get_data(post_list)
for each in data_list:
    print (each)

['[置顶]\n步行街主干道版规（严禁人肉、时政敏感帖，禁发募捐帖，切勿轻信他人，谨防诈骗）', 'https://bbs.hupu.com/15806550.html', '虎扑团队', 'https://my.hupu.com/272380999276581', '2017-01-18', '1', '3228499', '虎扑团队', datetime.date(2017, 2, 27)]
['抠图大师把底色换成蓝色，在车上公司要，没办法了', 'https://bbs.hupu.com/24555091.html', '沙棘汁', 'https://my.hupu.com/271275652956663', '2018-11-29', '1', '38', '把我的小熊', datetime.datetime(2018, 11, 29, 18, 14)]
['#被父母取名禤靐龘# 香港一初中生崩溃：同学喊我“喂”ZT\n\n\xa0\n\n[\xa0\n2\n3...7\n\n\xa0]', 'https://bbs.hupu.com/24554564.html', '晨灌吸', 'https://my.hupu.com/36713740238125', '2018-11-29', '122', '46656', 'o超级英雄o', datetime.datetime(2018, 11, 29, 18, 14)]
['男的真的很在意腿吗？\n\n\xa0\n\n[\xa0\n2\n3...22\n\n\xa0]', 'https://bbs.hupu.com/24554405.html', 'teihionn', 'https://my.hupu.com/170564828886404', '2018-11-29', '433', '119051', '天青色等烟俣丶', datetime.datetime(2018, 11, 29, 18, 14)]
['作为外地人，对青岛和济南是怎么看的呢？\n[\xa0\n2\n\n\xa0]', 'https://bbs.hupu.com/24552866.html', '20181015', 'https://my.hupu.com/244388484603367', '2018-11-29', '22', '10

In [8]:
from pymongo import MongoClient
class MongoAPI(object):
    def __init__(self, db_ip, db_port, db_name, table_name):
        self.db_ip = db_ip
        self.db_port = db_port
        self.db_name = db_name
        self.table_name = table_name
        self.conn = MongoClient(host=self.db_ip, port=self.db_port)
        self.db = self.conn[self.db_name]
        self.table = self.db[self.table_name]
    def get_one(self, query):
        return self.table.find_one(query, projection={"_id": False})
    def get_all(self, query):
        return self.table.find(query)
    def add(self, kv_dict):
        return self.table.insert(kv_dict)
    def delete(self, query):
        return self.table.delete_many(query)
    def check_exist(self, query):
        ret = self.table.find_one(query)
        return ret != None
    # 如果没有会新建
    def update(self, query, kv_dict):
            self.table.update_one(query,{
              '$set': kv_dict
            }, upsert=True)

In [9]:
hupu_post = MongoAPI("localhost",  27017,  "hupu", "post")
for each in data_list:
    hupu_post.add({"title": each[0], 
                "post_link": each[1],
               "author": each[2],
               "author_page": each[3],
               "start_date": str(each[4]),
               "reply": each[5],
               "view": each[6],
               "last_reply": each[7],
               "last_reply_time": str(each[8])})

C:\Users\santostang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  app.launch_new_instance()


In [18]:
import requests
from bs4 import BeautifulSoup
import datetime
from pymongo import MongoClient
import time

hupu_post = MongoAPI("localhost",  27017,  "hupu", "post")
for i in range(1,100):
    link = "https://bbs.hupu.com/bxj-" + str(i)
    soup = get_page(link)
    
    post_all= soup.find('ul', class_="for-list")
    post_list = post_all.find_all('li')
    data_list = get_data(post_list)
    for each in data_list:
        hupu_post.update({"post_link": each[1]},{"title": each[0], 
                    "post_link": each[1],
                   "author": each[2],
                   "author_page": each[3],
                   "start_date": str(each[4]),
                   "reply": each[5],
                   "view": each[6],
                   "last_reply": each[7],
                   "last_reply_time": str(each[8])})
    time.sleep(3)
    print ('第', i ,'页获取完成，休息3秒')

第 1 页获取完成，休息3秒
第 2 页获取完成，休息3秒
第 3 页获取完成，休息3秒
第 4 页获取完成，休息3秒
第 5 页获取完成，休息3秒
第 6 页获取完成，休息3秒
第 7 页获取完成，休息3秒
第 8 页获取完成，休息3秒
第 9 页获取完成，休息3秒
第 10 页获取完成，休息3秒


AttributeError: 'NoneType' object has no attribute 'find_all'